### 데이터 값 찾기

* 문서 URL 목록 가져오기

In [6]:
import pandas as pd

# 사업보고서 제무제표 주석 문서 URL 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/fs_notes_urls/fs_notes_url_all.csv"
fs_notes_all = pd.read_csv(file_path, dtype=object)
# 회사코드 정렬
fs_notes_all = fs_notes_all.sort_values(by=['corp_code'], ascending=True)
# 회사코드 목록
list_all_codes = fs_notes_all['corp_code'].unique()
len(list_all_codes)

342

* 이미 데이터 값을 찾은 것은 대상에서 제외

In [7]:
import os.path

# 이미 데이터 값을 찾은 회사코드 목록
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/items_all.csv"
 
if os.path.exists(file_path):
    try:
        items_all = pd.read_csv(file_path, dtype=object)
        list_own_codes = items_all['corp_code'].unique()
    except pd.errors.EmptyDataError:
        list_own_codes = []    
else:
    list_own_codes = []
    
len(list_own_codes)

242

In [8]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

100

* 리스트 분할

In [9]:
list_cnt = len(list_target_codes)
n = 5
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

20

* 항목과 값 찾기: 확정급여채무의 현재가치, 사외적립자산의 공정가치

In [5]:
import traceback
from time import sleep
from tqdm import tqdm

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil, mydart

# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/"

for corp_codes in list_of_lists:
    for corp_code in tqdm(corp_codes):
        str_expr = "corp_code == @corp_code"
        df_doc = fs_notes_all.query(str_expr)
        doc_count = df_doc.shape[0]
        data_list = []
        for i, doc in df_doc.iterrows():
            sleep(1.0)
            try:
                val_a = val_b = 0.0
                unit = None
                doc_result = doc['doc_result']
                if doc_result == 'No Financial Statement Notes':
                    data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                    'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                    'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : None, 
                                    'unit' : None, 'val_a' : None, 'val_b' : None, 'item_result' : None, 'doc_url' : None})
                else:
                    doc_url = doc['doc_url']                    
                    soup = mydart.get_document(doc_url)
                    subtitle_list = mydart.get_subtitles(soup)
                    section_title = mydart.find_section_title(subtitle_list)
                    if section_title is None:       # 해당 Section을 찾지 못했을 경우
                        data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                        'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : None, 
                                        'unit' : None, 'val_a' : None, 'val_b' : None, 'item_result' : None, 'doc_url' : doc_url})                        
                    else:                                         
                        section_page = mydart.get_section_html(soup, subtitle_list, section_title)
                        # 해당 Section에서 사용된 금액 단위
                        unit = mydart.find_unit_from_section(section_page)
                        # 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
                        val_a, val_b, item_result = mydart.find_item_from_section(section_page)      
                        data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                                        'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                                        'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : section_title, 
                                        'unit' : unit, 'val_a' : val_a, 'val_b' : val_b, 'item_result' : item_result, 'doc_url' : doc_url})
            except ValueError as err:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                print(err)
                break
            except Exception as e:
                print(doc['corp_code'], doc['corp_name'], doc['rpt_num'], doc['rpt_name'], doc['rpt_url'], doc['doc_title'], doc['doc_url'])
                err_msg = traceback.format_exc()
                print(err_msg)
                break
        # 결과를 파일로 저장
        doc_value_count = len(data_list)
        if doc_value_count == 0:
            break
        elif doc_value_count == doc_count:        
            df_items = pd.DataFrame(data_list)
            file_name = corp_code + '_item_' + myutil.now_dt_str() + '.csv'
            df_items.to_csv(path_dir + file_name, index=False)
        else:
            continue

 40%|████      | 2/5 [00:28<00:43, 14.45s/it]


00101044 에이프로젠바이오로직스 20230317000440 사업보고서 (2022.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230317000440 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20230317000440&dcmNo=9069444&eleId=22&offset=1001958&length=395281&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 801, in _build_doc
    r = parse(self.io, parser=parser)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\lxml\html\__init__.py", line 937, in parse
    return etree.parse(filename_or_url, parser, base_url=base_url, **kw)
  File "src\lxml\etree.pyx", line 3541, in lxml.etree.parse
  File "src\lxml\parser.pxi", line 1879, in lxml.etree._parseDocument
  File "src\lxml\parser.pxi", line 1905, in lxml.etree._parseDocumentFromURL
  File "src\lxml\parser.pxi", line 1808, in lxml.etree._parseDocFromFile
  File "src\lxml\parser.pxi", line 1180, in lxml.etree._BaseParser._parseDocFromFile
  

 20%|██        | 1/5 [00:12<00:50, 12.51s/it]


00137997 현대차증권 20230315000803 사업보고서 (2022.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230315000803 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20230315000803&dcmNo=9058745&eleId=20&offset=1388545&length=820300&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 801, in _build_doc
    r = parse(self.io, parser=parser)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\lxml\html\__init__.py", line 937, in parse
    return etree.parse(filename_or_url, parser, base_url=base_url, **kw)
  File "src\lxml\etree.pyx", line 3541, in lxml.etree.parse
  File "src\lxml\parser.pxi", line 1879, in lxml.etree._parseDocument
  File "src\lxml\parser.pxi", line 1905, in lxml.etree._parseDocumentFromURL
  File "src\lxml\parser.pxi", line 1808, in lxml.etree._parseDocFromFile
  File "src\lxml\parser.pxi", line 1180, in lxml.etree._BaseParser._parseDocFromFile
  File "

 20%|██        | 1/5 [00:17<01:09, 17.43s/it]

...

00129235 삼화왕관 20170330000136 사업보고서 (2016.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170330000136 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20170330000136&dcmNo=5515355&eleId=16&offset=215632&length=273820&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 801, in _build_doc
    r = parse(self.io, parser=parser)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\lxml\html\__init__.py", line 937, in parse
    return etree.parse(filename_or_url, parser, base_url=base_url, **kw)
  File "src\lxml\etree.pyx", line 3541, in lxml.etree.parse
  File "src\lxml\parser.pxi", line 1879, in lxml.etree._parseDocument
  File "src\lxml\parser.pxi", line 1905, in lxml.etree._parseDocumentFromURL
  File "src\lxml\parser.pxi", line 1808, in lxml.etree._parseDocFromFile
  File "src\lxml\parser.pxi", line 1180, in lxml.etree._BaseParser._parseDocFromFile
  File "sr

 25%|██▌       | 1/4 [00:14<00:42, 14.26s/it]

00159786 유니드 20180330001836 사업보고서 (2017.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20180330001836 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20180330001836&dcmNo=6033979&eleId=16&offset=904446&length=434934&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 801, in _build_doc
    r = parse(self.io, parser=parser)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\lxml\html\__init__.py", line 937, in parse
    return etree.parse(filename_or_url, parser, base_url=base_url, **kw)
  File "src\lxml\etree.pyx", line 3541, in lxml.etree.parse
  File "src\lxml\parser.pxi", line 1879, in lxml.etree._parseDocument
  File "src\lxml\parser.pxi", line 1905, in lxml.etree._parseDocumentFromURL
  File "src\lxml\parser.pxi", line 1808, in lxml.etree._parseDocFromFile
  File "src\lxml\parser.pxi", line 1180, in lxml.etree._BaseParser._parseDocFromFile
  File "src

 25%|██▌       | 1/4 [00:15<00:47, 16.00s/it]

01168383 일동제약 20170331004023 사업보고서 (2016.12) http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170331004023 5. 재무제표 주석 http://dart.fss.or.kr/report/viewer.do?rcpNo=20170331004023&dcmNo=5537786&eleId=16&offset=578548&length=238345&dtd=dart3.xsd
Traceback (most recent call last):
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\pandas\io\html.py", line 801, in _build_doc
    r = parse(self.io, parser=parser)
  File "d:\PythonProject\data-gatherer\data-gatherer.venv\lib\site-packages\lxml\html\__init__.py", line 937, in parse
    return etree.parse(filename_or_url, parser, base_url=base_url, **kw)
  File "src\lxml\etree.pyx", line 3541, in lxml.etree.parse
  File "src\lxml\parser.pxi", line 1879, in lxml.etree._parseDocument
  File "src\lxml\parser.pxi", line 1905, in lxml.etree._parseDocumentFromURL
  File "src\lxml\parser.pxi", line 1808, in lxml.etree._parseDocFromFile
  File "src\lxml\parser.pxi", line 1180, in lxml.etree._BaseParser._parseDocFromFile
  File "sr